In [1]:
import argparse
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, DatasetDict
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
import pandas as pd
import torch.utils.data as data
import torch
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm

device = "cuda"
model_name_or_path = "skt/ko-gpt-trinity-1.2B-v0.5"
tokenizer_name_or_path = "skt/ko-gpt-trinity-1.2B-v0.5"

checkpoint_path = '/home/yys/chatbot-paper/model_checkpoint/'
checkpoint_name = "ko-gpt-trinity-chatbot_lora_v1.pt"
text_column = "title"
label_column = "comment"
max_length = 50
lr = 1e-3
num_epochs = 1
batch_size = 4

In [2]:
model_name = 'skt/ko-gpt-trinity-1.2B-v0.5'
 
 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [6]:
# creating model
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)


model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 1474560 || all params: 1164030720 || trainable%: 0.12667706914126803


In [23]:
url = 'https://raw.githubusercontent.com/4N3MONE/COMU-ChatBot/main/data_final.tsv'
df = pd.read_csv(url, sep='\t')
df = df[['title','comment']]
dataset = Dataset.from_pandas(df)
dataset = DatasetDict({'train': dataset})
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]


dataset["train"][0]

{'title': '사랑받고 있다고 느낄때가 언제야?', 'comment': '그냥 대화하고 있는데 갑자기 볼뽀뽀할때!'}

In [25]:
# data preprocessing
def preprocess_function(examples):
    inputs = examples['title']
    targets = examples['comment']
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs



In [26]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/27690 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3077 [00:00<?, ? examples/s]

In [27]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [28]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)


In [29]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 770/770 [00:48<00:00, 15.91it/s]


epoch=0: train_ppl=tensor(2262.3906, device='cuda:0') train_epoch_loss=tensor(7.7242, device='cuda:0') eval_ppl=tensor(2240.9697, device='cuda:0') eval_epoch_loss=tensor(7.7147, device='cuda:0')


100%|██████████| 770/770 [00:47<00:00, 16.07it/s]


epoch=1: train_ppl=tensor(2429.7327, device='cuda:0') train_epoch_loss=tensor(7.7955, device='cuda:0') eval_ppl=tensor(2156.8333, device='cuda:0') eval_epoch_loss=tensor(7.6764, device='cuda:0')


100%|██████████| 770/770 [01:43<00:00,  7.44it/s]

epoch=2: train_ppl=tensor(2164.6936, device='cuda:0') train_epoch_loss=tensor(7.6800, device='cuda:0') eval_ppl=tensor(2123.5151, device='cuda:0') eval_epoch_loss=tensor(7.6608, device='cuda:0')


In [7]:
# model_dict = get_peft_model_state_dict(model)
# torch.save(model_dict, checkpoint_path+checkpoint_name)

In [9]:
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

In [10]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
ckpt = f"{peft_model_id}/adapter_model.bin"
!du -h $ckpt
config = PeftConfig.from_pretrained(peft_model_id)
model.save_pretrained(checkpoint_path)


du: cannot access ‘skt/ko-gpt-trinity-1.2B-v0.5_LORA_CAUSAL_LM/adapter_model.bin’: No such file or directory


ValueError: Can't find config.json at 'skt/ko-gpt-trinity-1.2B-v0.5_LORA_CAUSAL_LM'

In [52]:
model.eval()
i = 13
inputs = tokenizer(dataset["validation"][label_column][i], return_tensors="pt")
print(dataset["validation"][label_column][i])
print(inputs)

나같음 걍 ㅁㅈ 그럴때 곤란하긴 해 하고 말듯
{'input_ids': tensor([[29999, 18817, 25780,   565, 18869,   565,   477,   483, 32947, 21172,
         38473, 38165, 30033, 30356, 30023, 21079]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [109]:
model.eval()
i = 5
inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="pt")
inputs.to(device)
print(dataset["validation"][text_column][i])
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    outputs.to(device)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

ISTJ가 나 주려고 뭘 샀대
{'input_ids': tensor([[30551, 31785,   418, 18792, 29999, 30003, 30812, 34792, 40768, 20584]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
tensor([[30551, 31785,   418, 18792, 29999, 30003, 30812, 34792, 40768, 20584,
           565,   565,   565,   565,   565,   565,   565,   565,   565,   565]],
       device='cuda:0')
['ISTJ가 나 주려고 뭘 샀대          ']


In [104]:
inputs

{'input_ids': tensor([[42274, 24648, 30398, 25824, 23276, 30053, 39798, 36658, 30256, 35327,
         40355,   214]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}